In [75]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool, Value, Array
import time
from numba import njit
import numpy as np
import os
import multiprocessing
import random
import sys


In [61]:
def mc_pi(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [62]:
%%time

res = [mc_pi(int(1e6)) for i in range(2)]

KeyboardInterrupt: 

In [51]:
@njit()
def mc_pi_numba(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [52]:
%%time

res = [mc_pi_numba(int(1e6)) for i in range(2)]


CPU times: user 211 ms, sys: 7.98 ms, total: 219 ms
Wall time: 221 ms


In [53]:
np.array(res)

array([3.139124, 3.141576])

In [58]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e7) for i in range(1000)])

KeyboardInterrupt: 

In [55]:
np.array(list(res))

array([3.140996, 3.142452, 3.14368 , 3.14292 , 3.141316, 3.139808,
       3.139864, 3.140724, 3.142452, 3.139852])

In [63]:
%%time

with ProcessPoolExecutor(max_workers=20) as pool:
    res = pool.map(mc_pi_numba, [int(1e4) for i in range(int(1e3))])

CPU times: user 1.26 s, sys: 140 ms, total: 1.4 s
Wall time: 2.59 s


In [57]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e4) for i in range(int(1e3))], chunksize=100)

CPU times: user 20.9 ms, sys: 24 ms, total: 45 ms
Wall time: 137 ms


In [66]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e7) for i in range(10)])

CPU times: user 26 ms, sys: 16 ms, total: 42 ms
Wall time: 754 ms


In [68]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e4) for i in range(int(1e4))])

CPU times: user 27.7 ms, sys: 40 ms, total: 67.8 ms
Wall time: 864 ms


In [69]:
def f(i):
    time.sleep(np.random.random())
    print(os.getpid(), i)

In [73]:
for i in range(10):
    p = mp.Process(target=f, args=(i,))
    p.start()
    p.join()

31177 0
31186 1
31197 2
31206 3
31215 4
31224 5
31233 6
31242 7
31252 8
31262 9


In [98]:
#
# Functions used by test code
#

def calculate(func, args):
    result = func(*args)
    return '%s says that %s%s = %s' % (
        multiprocessing.current_process().name,
        func.__name__, args, result
        )

def calculatestar(args):
    return calculate(*args)

def mul(a, b):
    time.sleep(0.5 * random.random())
    return a * b

def plus(a, b):
    time.sleep(0.5 * random.random())
    return a + b

def f(x):
    return 1.0 / (x - 5.0)

def pow3(x):
    return x ** 3

def noop(x):
    pass

def test():
    
    for j in range(6):
        PROCESSES = int(j+1) 
        print(PROCESSES)
        print('Creating pool with %d processes\n' % PROCESSES)

        test = np.random.rand(int(10),1)

        with multiprocessing.Pool(PROCESSES) as pool:

            #
            # Tests
            #

            TASKS = [(mul, (test[i], 7)) for i in range(len(test))] + \
                    [(plus, (test[i], 8)) for i in range(len(test))]

            results = [pool.apply_async(calculate, t) for t in TASKS]
            imap_it = pool.imap(calculatestar, TASKS)
            imap_unordered_it = pool.imap_unordered(calculatestar, TASKS)
            
            print(imap_it)

            print('Ordered results using pool.apply_async():')
            for r in results:
                print('\t', r.get())
            print()

            print()
            for x in imap_it:
                print(type(imap_it))
                print(x)
            print()

            print('Unordered results using pool.imap_unordered():')
            for x in imap_unordered_it:
                print('\t', x)
            print()

            print('Ordered results using pool.map() --- will block till complete:')
            for x in pool.map(calculatestar, TASKS):
                print('\t', x)
            print()

            #
            # Test error handling
            #

            print('Testing error handling:')

            try:
                print(pool.apply(f, (5,)))
            except ZeroDivisionError:
                print('\tGot ZeroDivisionError as expected from pool.apply()')
            else:
                raise AssertionError('expected ZeroDivisionError')

            try:
                print(pool.map(f, list(range(10))))
            except ZeroDivisionError:
                print('\tGot ZeroDivisionError as expected from pool.map()')
            else:
                raise AssertionError('expected ZeroDivisionError')

            try:
                print(list(pool.imap(f, list(range(10)))))
            except ZeroDivisionError:
                print('\tGot ZeroDivisionError as expected from list(pool.imap())')
            else:
                raise AssertionError('expected ZeroDivisionError')

            it = pool.imap(f, list(range(10)))
            for i in range(10):
                try:
                    x = next(it)
                except ZeroDivisionError:
                    if i == 5:
                        pass
                except StopIteration:
                    break
                else:
                    if i == 5:
                        raise AssertionError('expected ZeroDivisionError')

            assert i == 9
            print('\tGot ZeroDivisionError as expected from IMapIterator.next()')
            print()

            #
            # Testing timeouts
            #

            print('Testing ApplyResult.get() with timeout:', end=' ')
            res = pool.apply_async(calculate, TASKS[0])
            while 1:
                sys.stdout.flush()
                try:
                    sys.stdout.write('\n\t%s' % res.get(0.02))
                    break
                except multiprocessing.TimeoutError:
                    sys.stdout.write('.')
            print()
            print()

            print('Testing IMapIterator.next() with timeout:', end=' ')
            it = pool.imap(calculatestar, TASKS)
            while 1:
                sys.stdout.flush()
                try:
                    sys.stdout.write('\n\t%s' % it.next(0.02))
                except StopIteration:
                    break
                except multiprocessing.TimeoutError:
                    sys.stdout.write('.')
            print()
            print()


if __name__ == '__main__':
    multiprocessing.freeze_support()
    test()

1
Creating pool with 1 processes

Ordered results using pool.apply_async():
	 ForkPoolWorker-258 says that mul(array([0.89492047]), 7) = [6.26444327]
	 ForkPoolWorker-258 says that mul(array([0.36236341]), 7) = [2.53654387]
	 ForkPoolWorker-258 says that mul(array([0.70704634]), 7) = [4.94932438]
	 ForkPoolWorker-258 says that mul(array([0.32468961]), 7) = [2.27282728]
	 ForkPoolWorker-258 says that mul(array([0.98378638]), 7) = [6.88650466]
	 ForkPoolWorker-258 says that mul(array([0.19568356]), 7) = [1.36978493]
	 ForkPoolWorker-258 says that mul(array([0.87508434]), 7) = [6.1255904]
	 ForkPoolWorker-258 says that mul(array([0.39902321]), 7) = [2.79316248]
	 ForkPoolWorker-258 says that mul(array([0.22638937]), 7) = [1.58472556]
	 ForkPoolWorker-258 says that mul(array([0.86769808]), 7) = [6.07388655]
	 ForkPoolWorker-258 says that plus(array([0.89492047]), 8) = [8.89492047]
	 ForkPoolWorker-258 says that plus(array([0.36236341]), 8) = [8.36236341]
	 ForkPoolWorker-258 says that plus

	ForkPoolWorker-258 says that mul(array([0.22638937]), 7) = [1.58472556].................
	ForkPoolWorker-258 says that mul(array([0.86769808]), 7) = [6.07388655]......
	ForkPoolWorker-258 says that plus(array([0.89492047]), 8) = [8.89492047]................
	ForkPoolWorker-258 says that plus(array([0.36236341]), 8) = [8.36236341].........
	ForkPoolWorker-258 says that plus(array([0.70704634]), 8) = [8.70704634]...................
	ForkPoolWorker-258 says that plus(array([0.32468961]), 8) = [8.32468961].
	ForkPoolWorker-258 says that plus(array([0.98378638]), 8) = [8.98378638].....................
	ForkPoolWorker-258 says that plus(array([0.19568356]), 8) = [8.19568356]......
	ForkPoolWorker-258 says that plus(array([0.87508434]), 8) = [8.87508434]..
	ForkPoolWorker-258 says that plus(array([0.39902321]), 8) = [8.39902321]....
	ForkPoolWorker-258 says that plus(array([0.22638937]), 8) = [8.22638937].....................
	ForkPoolWorker-258 says that plus(array([0.86769808]), 8) = [8.86

Testing ApplyResult.get() with timeout: .......
	ForkPoolWorker-259 says that mul(array([0.0262051]), 7) = [0.18343571]

Testing IMapIterator.next() with timeout: .........
	ForkPoolWorker-260 says that mul(array([0.0262051]), 7) = [0.18343571].
	ForkPoolWorker-259 says that mul(array([0.55690306]), 7) = [3.89832142].........
	ForkPoolWorker-260 says that mul(array([0.72391239]), 7) = [5.06738672]
	ForkPoolWorker-259 says that mul(array([0.60415414]), 7) = [4.22907895]..........
	ForkPoolWorker-259 says that mul(array([0.20325791]), 7) = [1.42280536]......
	ForkPoolWorker-260 says that mul(array([0.54811446]), 7) = [3.83680124]..........
	ForkPoolWorker-259 says that mul(array([0.64034807]), 7) = [4.48243648].
	ForkPoolWorker-260 says that mul(array([0.25258942]), 7) = [1.76812596]..........
	ForkPoolWorker-259 says that mul(array([0.08429628]), 7) = [0.59007399]
	ForkPoolWorker-260 says that mul(array([0.34011452]), 7) = [2.38080161]
	ForkPoolWorker-260 says that plus(array([0.0262051

Testing ApplyResult.get() with timeout: ..............
	ForkPoolWorker-261 says that mul(array([0.04380844]), 7) = [0.30665908]

Testing IMapIterator.next() with timeout: ..
	ForkPoolWorker-262 says that mul(array([0.04380844]), 7) = [0.30665908]
	ForkPoolWorker-263 says that mul(array([0.05217814]), 7) = [0.36524697]....
	ForkPoolWorker-261 says that mul(array([0.55770481]), 7) = [3.90393367]
	ForkPoolWorker-262 says that mul(array([0.26519634]), 7) = [1.85637439]
	ForkPoolWorker-263 says that mul(array([0.91450948]), 7) = [6.40156638].........
	ForkPoolWorker-263 says that mul(array([0.63288734]), 7) = [4.43021136]
	ForkPoolWorker-262 says that mul(array([0.84800903]), 7) = [5.93606322]
	ForkPoolWorker-261 says that mul(array([0.97176394]), 7) = [6.80234759].....
	ForkPoolWorker-263 says that mul(array([0.39617695]), 7) = [2.77323864]
	ForkPoolWorker-262 says that mul(array([0.04455746]), 7) = [0.31190222]...
	ForkPoolWorker-261 says that plus(array([0.04380844]), 8) = [8.04380844]..

Testing ApplyResult.get() with timeout: .........
	ForkPoolWorker-265 says that mul(array([0.76272544]), 7) = [5.33907806]

Testing IMapIterator.next() with timeout: .........
	ForkPoolWorker-266 says that mul(array([0.76272544]), 7) = [5.33907806]
	ForkPoolWorker-264 says that mul(array([0.2849875]), 7) = [1.99491251]
	ForkPoolWorker-267 says that mul(array([0.10491153]), 7) = [0.73438072]..
	ForkPoolWorker-265 says that mul(array([0.51973138]), 7) = [3.63811965].....
	ForkPoolWorker-264 says that mul(array([0.14719278]), 7) = [1.03034944]
	ForkPoolWorker-267 says that mul(array([0.83713844]), 7) = [5.85996909]..
	ForkPoolWorker-266 says that mul(array([0.64988841]), 7) = [4.54921885]........
	ForkPoolWorker-265 says that mul(array([0.71025714]), 7) = [4.97179995]
	ForkPoolWorker-267 says that mul(array([0.6514799]), 7) = [4.56035931]
	ForkPoolWorker-264 says that mul(array([0.70339703]), 7) = [4.92377919]........
	ForkPoolWorker-266 says that plus(array([0.76272544]), 8) = [8.7627254

Testing ApplyResult.get() with timeout: .
	ForkPoolWorker-271 says that mul(array([0.10581378]), 7) = [0.74069648]

Testing IMapIterator.next() with timeout: .
	ForkPoolWorker-270 says that mul(array([0.10581378]), 7) = [0.74069648]
	ForkPoolWorker-272 says that mul(array([0.92241992]), 7) = [6.45693944]
	ForkPoolWorker-269 says that mul(array([0.52727645]), 7) = [3.69093517]
	ForkPoolWorker-268 says that mul(array([0.81383623]), 7) = [5.69685364]....................
	ForkPoolWorker-271 says that mul(array([0.91258466]), 7) = [6.38809263].
	ForkPoolWorker-270 says that mul(array([0.70362293]), 7) = [4.92536048]
	ForkPoolWorker-272 says that mul(array([0.41623771]), 7) = [2.91366399]
	ForkPoolWorker-268 says that mul(array([0.95239861]), 7) = [6.66679029]
	ForkPoolWorker-269 says that mul(array([0.16160824]), 7) = [1.13125769]...........
	ForkPoolWorker-271 says that mul(array([0.72157681]), 7) = [5.05103764]
	ForkPoolWorker-270 says that plus(array([0.10581378]), 8) = [8.10581378]
	For

Testing ApplyResult.get() with timeout: ................
	ForkPoolWorker-274 says that mul(array([0.40112486]), 7) = [2.80787401]

Testing IMapIterator.next() with timeout: .....
	ForkPoolWorker-276 says that mul(array([0.40112486]), 7) = [2.80787401]..........
	ForkPoolWorker-278 says that mul(array([0.01158754]), 7) = [0.08111281]
	ForkPoolWorker-277 says that mul(array([0.65167023]), 7) = [4.56169158]
	ForkPoolWorker-275 says that mul(array([0.16857863]), 7) = [1.18005038]
	ForkPoolWorker-273 says that mul(array([0.37914614]), 7) = [2.654023]
	ForkPoolWorker-274 says that mul(array([0.99186707]), 7) = [6.94306952].....
	ForkPoolWorker-276 says that mul(array([0.11103213]), 7) = [0.77722493]
	ForkPoolWorker-277 says that mul(array([0.00449844]), 7) = [0.0314891]
	ForkPoolWorker-273 says that mul(array([0.59366093]), 7) = [4.15562649]
	ForkPoolWorker-275 says that mul(array([0.29400359]), 7) = [2.05802516]
	ForkPoolWorker-274 says that plus(array([0.40112486]), 8) = [8.40112486]......

In [100]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

if __name__ == '__main__':
    # start 4 worker processes
    with Pool(processes=4) as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(20)" asynchronously
        res = pool.apply_async(f, (20,))      # runs in *only* one process
        print(res.get(timeout=1))             # prints "400"

        # evaluate "os.getpid()" asynchronously
        res = pool.apply_async(os.getpid, ()) # runs in *only* one process
        print(res.get(timeout=1))             # prints the PID of that process

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 secs
        res = pool.apply_async(time.sleep, (10,))
        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing.TimeoutError")

        print("For the moment, the pool remains available for more work")

    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
4
9
16
25
36
64
49
81
400
33257
[33257, 33258, 33260, 33259]
We lacked patience and got a multiprocessing.TimeoutError
For the moment, the pool remains available for more work
Now the pool is closed and no longer available
